from https://github.com/huggingface/lerobot/blob/main/examples/1_load_lerobot_dataset.py

In [3]:
"""
This script demonstrates the use of `LeRobotDataset` class for handling and processing robotic datasets from Hugging Face.
It illustrates how to load datasets, manipulate them, and apply transformations suitable for machine learning tasks in PyTorch.

Features included in this script:
- Viewing a dataset's metadata and exploring its properties.
- Loading an existing dataset from the hub or a subset of it.
- Accessing frames by episode number.
- Using advanced dataset features like timestamp-based frame selection.
- Demonstrating compatibility with PyTorch DataLoader for batch processing.

The script ends with examples of how to batch process data using PyTorch's DataLoader.
"""

from pprint import pprint


from lerobot.common.datasets.lerobot_dataset import LeRobotDatasetMetadata

# Or simply explore them in your web browser directly at:
# https://huggingface.co/datasets?other=LeRobot

# Let's take this one for this example
repo_id = "Haskely/exam_2"
kwargs = dict(
    root="../data/dataset/exam_2",
    local_files_only=True,
)
# We can have a look and fetch its metadata to know more about it:
ds_meta = LeRobotDatasetMetadata(repo_id, **kwargs)

# By instantiating just this class, you can quickly access useful information about the content and the
# structure of the dataset without downloading the actual data yet (only metadata files — which are
# lightweight).
print(f"Total number of episodes: {ds_meta.total_episodes}")
print(
    f"Average number of frames per episode: {ds_meta.total_frames / ds_meta.total_episodes:.3f}"
)
print(f"Frames per second used during data collection: {ds_meta.fps}")
print(f"Robot type: {ds_meta.robot_type}")
print(f"keys to access images from cameras: {ds_meta.camera_keys=}\n")

print("Tasks:")
print(ds_meta.tasks)
print("Features:")
pprint(ds_meta.features)

# You can also get a short summary by simply printing the object:
print(ds_meta)

Returning existing local_dir `..\data\dataset\exam_2` as remote repo cannot be accessed in `snapshot_download` (None).


Total number of episodes: 10
Average number of frames per episode: 1436.300
Frames per second used during data collection: 30
Robot type: koch
keys to access images from cameras: ds_meta.camera_keys=['observation.images.top', 'observation.images.side', 'observation.images.arm']

Tasks:
{0: '将篮子放到容器里'}
Features:
{'action': {'dtype': 'float32',
            'names': ['main_shoulder_pan',
                      'main_shoulder_lift',
                      'main_elbow_flex',
                      'main_wrist_flex',
                      'main_wrist_roll',
                      'main_gripper'],
            'shape': (6,)},
 'episode_index': {'dtype': 'int64', 'names': None, 'shape': (1,)},
 'frame_index': {'dtype': 'int64', 'names': None, 'shape': (1,)},
 'index': {'dtype': 'int64', 'names': None, 'shape': (1,)},
 'observation.images.arm': {'dtype': 'video',
                            'info': {'has_audio': False,
                                     'video.channels': 3,
                       

In [6]:
# You can then load the actual dataset from the hub.
# Either load any subset of episodes:
from lerobot.common.datasets.lerobot_dataset import LeRobotDataset


dataset = LeRobotDataset(repo_id, episodes=[0, 7], **kwargs)

# And see how many frames you have:
print(f"Selected episodes: {dataset.episodes}")
print(f"Number of episodes selected: {dataset.num_episodes}")
print(f"Number of frames selected: {dataset.num_frames}")

# Or simply load the entire dataset:
dataset = LeRobotDataset(repo_id, **kwargs)
print(f"Number of episodes selected: {dataset.num_episodes}")
print(f"Number of frames selected: {dataset.num_frames}")

# The previous metadata class is contained in the 'meta' attribute of the dataset:
print(dataset.meta)

# LeRobotDataset actually wraps an underlying Hugging Face dataset
# (see https://huggingface.co/docs/datasets for more information).
print(dataset.hf_dataset)

Returning existing local_dir `..\data\dataset\exam_2` as remote repo cannot be accessed in `snapshot_download` (None).
Returning existing local_dir `..\data\dataset\exam_2` as remote repo cannot be accessed in `snapshot_download` (None).
Returning existing local_dir `..\data\dataset\exam_2` as remote repo cannot be accessed in `snapshot_download` (None).
Returning existing local_dir `..\data\dataset\exam_2` as remote repo cannot be accessed in `snapshot_download` (None).


Selected episodes: [0, 7]
Number of episodes selected: 2
Number of frames selected: 3560
Number of episodes selected: 10
Number of frames selected: 14363
LeRobotDatasetMetadata({
    Repository ID: 'Haskely/exam_2',
    Total episodes: '10',
    Total frames: '14363',
    Features: '['action', 'observation.state', 'observation.images.top', 'observation.images.side', 'observation.images.arm', 'timestamp', 'frame_index', 'episode_index', 'index', 'task_index']',
})',

Dataset({
    features: ['action', 'observation.state', 'timestamp', 'frame_index', 'episode_index', 'index', 'task_index'],
    num_rows: 14363
})


In [8]:
dataset.episode_data_index

{'from': tensor([    0,  1953,  3364,  4692,  6032,  7266,  9015, 10550, 12157, 13463]),
 'to': tensor([ 1953,  3364,  4692,  6032,  7266,  9015, 10550, 12157, 13463, 14363])}

In [ ]:
episode_index = 0
from_idx = dataset.episode_data_index["from"][episode_index].item()
to_idx = dataset.episode_data_index["to"][episode_index].item()

In [11]:
dataset.meta.features

{'action': {'dtype': 'float32',
  'shape': (6,),
  'names': ['main_shoulder_pan',
   'main_shoulder_lift',
   'main_elbow_flex',
   'main_wrist_flex',
   'main_wrist_roll',
   'main_gripper']},
 'observation.state': {'dtype': 'float32',
  'shape': (6,),
  'names': ['main_shoulder_pan',
   'main_shoulder_lift',
   'main_elbow_flex',
   'main_wrist_flex',
   'main_wrist_roll',
   'main_gripper']},
 'observation.images.top': {'dtype': 'video',
  'shape': (480, 640, 3),
  'names': ['height', 'width', 'channels'],
  'info': {'video.fps': 30.0,
   'video.height': 480,
   'video.width': 640,
   'video.channels': 3,
   'video.codec': 'av1',
   'video.pix_fmt': 'yuv420p',
   'video.is_depth_map': False,
   'has_audio': False}},
 'observation.images.side': {'dtype': 'video',
  'shape': (480, 640, 3),
  'names': ['height', 'width', 'channels'],
  'info': {'video.fps': 30.0,
   'video.height': 480,
   'video.width': 640,
   'video.channels': 3,
   'video.codec': 'av1',
   'video.pix_fmt': 'yuv420p

In [ ]:
# LeRobot datasets also subclasses PyTorch datasets so you can do everything you know and love from working
# with the latter, like iterating through the dataset.
# The __getitem__ iterates over the frames of the dataset. Since our datasets are also structured by
# episodes, you can access the frame indices of any episode using the episode_data_index. Here, we access
# frame indices associated to the first episode:
episode_index = 0
from_idx = dataset.episode_data_index["from"][episode_index].item()
to_idx = dataset.episode_data_index["to"][episode_index].item()

# Then we grab all the image frames from the first camera:
camera_key = dataset.meta.camera_keys[0]
frames = [dataset[idx][camera_key] for idx in range(from_idx, to_idx)]

# The objects returned by the dataset are all torch.Tensors
print(type(frames[0]))
print(frames[0].shape)

# Since we're using pytorch, the shape is in pytorch, channel-first convention (c, h, w).
# We can compare this shape with the information available for that feature
pprint(dataset.features[camera_key])
# In particular:
print(dataset.features[camera_key]["shape"])
# The shape is in (h, w, c) which is a more universal format.